In [1]:
%matplotlib inline
import pymongo
import tweepy
import networkx as nx

from pymongo import MongoClient

from sweepy.get_config import get_config

config = get_config()

MONGO_URL = config.get('MONGO_URL')
MONGO_PORT = config.get('MONGO_PORT')
MONGO_USERNAME = config.get('MONGO_USERNAME')
MONGO_PASSWORD = config.get('MONGO_PASSWORD')

client = MongoClient(MONGO_URL, int(MONGO_PORT))

db = client.tweets
db.authenticate(MONGO_USERNAME, MONGO_PASSWORD)

users = db.users

In [2]:
graph  = nx.DiGraph()

In [3]:
for user in users.find():
    graph.add_node(user['id_str'])
    for friend_id in user['friends_ids']:
        if not friend_id in graph:
            graph.add_node(friend_id)
        graph.add_edge(user['id_str'], friend_id)

In [4]:
graph.number_of_edges()

5248563

In [5]:
graph.number_of_nodes()

2421125

In [ ]:
nx.betweenness_centrality(graph)

In [ ]:
nx.draw(graph)